In [1]:
# imports
import requests
import json
import pandas as pd
import os

In [2]:
bike_stations_df = pd.read_csv('bike_stations.csv', index_col = 0)

In [3]:
bike_stations_df

,Station Name,Available Bikes,Empty Slots,Latitude,Longitude
0,Gare d'autocars de Montréal (Berri / Ontario),5,10,45.516926,-73.564257
1,Marché Maisonneuve,7,15,45.553219,-73.539782
2,Rachel / de Brébeuf,24,5,45.526890,-73.572640
3,Bibliothèque d'Ahuntsic (Lajeunesse / Fleury),8,6,45.553400,-73.662255
4,Cité des Arts du Cirque (Paul Boutet / des Reg...,22,12,45.559842,-73.615447
...,...,...,...,...,...
155,Métro Atwater (Atwater / Ste-Catherine),19,16,45.489525,-73.584458
156,Complexe Desjardins (St-Urbain / René-Lévesque),8,32,45.507885,-73.563151
157,de Maisonneuve / Mackay,6,13,45.496496,-73.578704
158,du Mont-Royal / de Brébeuf,10,1,45.529337,-73.577953


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [26]:
def query_foursquare_for_places(lat, lng, radius=1000, limit = 50):
    url = f"https://api.foursquare.com/v3/places/search"
    FOURSQUARE_API_VERSION = '20230820'
    params = {
        'v': FOURSQUARE_API_VERSION,
        'll': f"{lat},{lng}",
        'radius': radius
    }
    
    api_key = os.getenv("FOURSQUARE_API_KEY")
    
    # Create dictionary for headers
    headers = {"Accept": "application/json"}
    # Add key with our API KEY
    headers['Authorization'] = api_key

    try:
        response = requests.get(url, headers=headers, params=params)
        #print(response)
        #print("Response content:", response.content)  # Print the raw response content
        response_data = response.json()
        #print(response.json())
        
        if 'results' in response_data:
            places = response_data['results']
            return places
        else:
            print("No Place of Interest found")
            return []         

    except requests.exceptions.RequestException as e:
        print("Error making API request:", e)
        return []

In [27]:
if bike_stations_df is not None:
    for index, row in bike_stations_df.iterrows():
        print(f"Station Name: {row['Station Name']}")
        print(f"Available Bikes: {row['Available Bikes']}")
        print(f"Empty Slots: {row['Empty Slots']}")
        print(f"Latitude: {row['Latitude']}")
        print(f"Longitude: {row['Longitude']}")
        
        query_foursquare_for_places(row['Latitude'], row['Longitude'])

Station Name: Gare d'autocars de Montréal (Berri / Ontario)
Available Bikes: 5
Empty Slots: 10
Latitude: 45.516926210319546
Longitude: -73.56425732374191
No Place of Interest found
Station Name: Marché Maisonneuve
Available Bikes: 7
Empty Slots: 15
Latitude: 45.55321884238814
Longitude: -73.53978216648102
No Place of Interest found
Station Name: Rachel / de Brébeuf
Available Bikes: 24
Empty Slots: 5
Latitude: 45.52689
Longitude: -73.57264
No Place of Interest found
Station Name: Bibliothèque d'Ahuntsic (Lajeunesse / Fleury)
Available Bikes: 8
Empty Slots: 6
Latitude: 45.5534000891078
Longitude: -73.66225451231003
No Place of Interest found
Station Name: Cité des Arts du Cirque (Paul Boutet / des Regrattiers)
Available Bikes: 22
Empty Slots: 12
Latitude: 45.55984236120471
Longitude: -73.61544728279114
No Place of Interest found
Station Name: Omer-Lavallée / du Midway
Available Bikes: 7
Empty Slots: 10
Latitude: 45.5457759528664
Longitude: -73.56217458844185
No Place of Interest found
St

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [10]:
def foursqare_places_of_interest(lat, lng, limit = 50):
    poi = query_foursquare_for_places(lat, lng, radius=1000)
    
    if poi:
        print("Nearby Places of Interest:")
        for places in poi:
            if 'coffee' in places['categories'][0]['name'].lower() or 'restaurant' in places['categories'][0]['name'].lower() or 'bar' in places['categories'][0]['name'].lower() or 'hotel' in places['categories'][0]['name'].lower():
                print(f"Business Category: {places['categories'][0]['name']}")
                if places['chains']:
                    print(f"Business Chain: {places['chains']}")
                print(f"Business Name: {places['name']}")
                print(f"Business Address: {places['location']['formatted_address']}")
                print("-" * 30)
    else:
        print("No Business found")

In [11]:
if bike_stations_df is not None:
    for index, row in bike_stations_df.iterrows():
        print(f"Station Name: {row['Station Name']}")
        print(f"Available Bikes: {row['Available Bikes']}")
        print(f"Empty Slots: {row['Empty Slots']}")
        print(f"Latitude: {row['Latitude']}")
        print(f"Longitude: {row['Longitude']}")
        
        
        foursqare_places_of_interest(row['Latitude'], row['Longitude'])

Station Name: Gare d'autocars de Montréal (Berri / Ontario)
Available Bikes: 5
Empty Slots: 10
Latitude: 45.516926210319546
Longitude: -73.56425732374191
Nearby Places of Interest:
Business Category: Bar
Business Name: Le Cheval Blanc
Business Address: 809 Ontario Rue E (entre St-Hubert & St-Christophe), Montréal QC H2L 1P1
------------------------------
Business Category: Sushi Restaurant
Business Name: Vua Sandwichs
Business Address: 1579 Saint-Denis Rue (entre De Maisonneuve & Émery), Montréal QC H2X 3K3
------------------------------
Business Category: Restaurant
Business Name: Le Mousso
Business Address: 1023 Ontario Rue E (entre Amherst & St-Timothée), Montréal QC H2L 1P8
------------------------------
Station Name: Marché Maisonneuve
Available Bikes: 7
Empty Slots: 15
Latitude: 45.55321884238814
Longitude: -73.53978216648102
Nearby Places of Interest:
Business Category: Middle Eastern Restaurant
Business Name: L'Olive Noire
Business Address: 4271 Ontario Rue E, Montréal QC H1V 1

IndexError: list index out of range

Put your parsed results into a DataFrame

In [12]:
business = []

In [14]:
if bike_stations_df is not None:
    for index, row in bike_stations_df.iterrows():
        results = query_foursquare_for_places(row['Latitude'], row['Longitude'], limit = 50)
        
        for each in results:
            if each['categories']:
                Category = each['categories'][0]['name']
            if each['name']:
                Name = each['name']
            if each['location']:
                Location = each['location']['formatted_address']
            
                      
            if 'coffee' in Category.lower() or 'restaurant' in Category.lower() or 'bar' in Category.lower() or 'hotel' in Category.lower():
                business.append({'Station Name': row['Station Name'], 'Business Category': Category, 'Business Name': Name, 'Address': Location})

In [15]:
foursquare_business_df = pd.DataFrame(business)
foursquare_business_df

,Station Name,Business Category,Business Name,Address
0,Gare d'autocars de Montréal (Berri / Ontario),Bar,Le Cheval Blanc,809 Ontario Rue E (entre St-Hubert & St-Christ...
1,Gare d'autocars de Montréal (Berri / Ontario),Sushi Restaurant,Vua Sandwichs,1579 Saint-Denis Rue (entre De Maisonneuve & É...
2,Gare d'autocars de Montréal (Berri / Ontario),Restaurant,Le Mousso,1023 Ontario Rue E (entre Amherst & St-Timothé...
3,Marché Maisonneuve,Middle Eastern Restaurant,L'Olive Noire,"4271 Ontario Rue E, Montréal QC H1V 1K4"
4,Cité des Arts du Cirque (Paul Boutet / des Reg...,Vietnamese Restaurant,Pho Ngon,"3205 Cremazie Blvd E (Crémazie), Montréal QC H..."
...,...,...,...,...
423,de Maisonneuve / Mackay,Salad Restaurant,Mandy's,"2067 Crescent Rue, Montréal QC H3G 2C1"
424,de Maisonneuve / Mackay,Italian Restaurant,Ristorante Beatrice,"1504 Sherbrooke Rue O, Montréal QC H3G 1L3"
425,du Mont-Royal / de Brébeuf,Coffee Shop,Café Myriade - le Plateau,"4627 Saint-Denis Rue, Montréal QC H2J 2L4"
426,Ste-Catherine / Union,Cocktail Bar,Nacarat,"900 René-Lévesque O, Montréal QC H3B 4A5"


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [29]:
def query_yelp_for_places(lat, lng, radius = 1000, limit = 10):
    url = "https://api.yelp.com/v3/businesses/search"
    api_key = os.getenv("YELP_API_KEY")
    params = {
        "latitude": lat,
        "longitude": lng,
        "radius": radius
    }
    
    headers = {
        "Authorization": f"Bearer {api_key}"
    }
    
    try:
        response = requests.get(url, params=params, headers=headers)
        #print(response)
        #print(response.content)
        response_data= response.json()
        
        if 'businesses' in response_data:
            places = response_data['businesses']
            return places
        else:
            print("No Place of Interest found")
            return []
    except requests.exceptions.RequestException as e:
        print("Error making API request:", e)
        return []

In [30]:
if bike_stations_df is not None:
    for index, row in bike_stations_df.iterrows():
        print(f"Station Name: {row['Station Name']}")
        print(f"Available Bikes: {row['Available Bikes']}")
        print(f"Empty Slots: {row['Empty Slots']}")
        print(f"Latitude: {row['Latitude']}")
        print(f"Longitude: {row['Longitude']}")
        
        query_yelp_for_places(row['Latitude'], row['Longitude'])

Station Name: Gare d'autocars de Montréal (Berri / Ontario)
Available Bikes: 5
Empty Slots: 10
Latitude: 45.516926210319546
Longitude: -73.56425732374191
No Place of Interest found
Station Name: Marché Maisonneuve
Available Bikes: 7
Empty Slots: 15
Latitude: 45.55321884238814
Longitude: -73.53978216648102
No Place of Interest found
Station Name: Rachel / de Brébeuf
Available Bikes: 24
Empty Slots: 5
Latitude: 45.52689
Longitude: -73.57264
No Place of Interest found
Station Name: Bibliothèque d'Ahuntsic (Lajeunesse / Fleury)
Available Bikes: 8
Empty Slots: 6
Latitude: 45.5534000891078
Longitude: -73.66225451231003
No Place of Interest found
Station Name: Cité des Arts du Cirque (Paul Boutet / des Regrattiers)
Available Bikes: 22
Empty Slots: 12
Latitude: 45.55984236120471
Longitude: -73.61544728279114
No Place of Interest found
Station Name: Omer-Lavallée / du Midway
Available Bikes: 7
Empty Slots: 10
Latitude: 45.5457759528664
Longitude: -73.56217458844185
No Place of Interest found
St

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [18]:
def yelp_places_of_interest(lat, lng, limit = 50):
    poi = query_yelp_for_places(lat, lng, radius = 1000, limit = 50)
    
    if poi:
        print("Nearby Places of Interest:")
        for places in poi:
            print(f"Business Category: {places['categories'][0]['title']}")
            print(f"Business Name: {places['name']}")
            print(f"No of Reviews: {places['review_count']}")
            print(f"Rating: {places['rating']}")
            print(f"Business Address: {places['location']['display_address'][0]}")
            print(f"Business Contact: {places['display_phone']}")
            print("-" * 30)
    else:
        print("No Business found")

In [19]:

if bike_stations_df is not None:
    for index, row in bike_stations_df.iterrows():
        print(f"Station Name: {row['Station Name']}")
        print(f"Available Bikes: {row['Available Bikes']}")
        print(f"Empty Slots: {row['Empty Slots']}")
        print(f"Latitude: {row['Latitude']}")
        print(f"Longitude: {row['Longitude']}")
        
        yelp_places_of_interest(row['Latitude'], row['Longitude'])

Station Name: Gare d'autocars de Montréal (Berri / Ontario)
Available Bikes: 5
Empty Slots: 10
Latitude: 45.516926210319546
Longitude: -73.56425732374191
Nearby Places of Interest:
Business Category: Brasseries
Business Name: Le Saint-Bock
No of Reviews: 208
Rating: 4.0
Business Address: 1749 Rue Saint-Denis
Business Contact: +1 514-680-8052
------------------------------
Business Category: Tapas Bars
Business Name: L'Amère à Boire
No of Reviews: 68
Rating: 4.0
Business Address: 2049 Rue Saint-Denis
Business Contact: +1 514-282-7448
------------------------------
Business Category: Pizza
Business Name: Pizzeria Dei Compari
No of Reviews: 91
Rating: 4.0
Business Address: 1668 Rue Saint-Denis
Business Contact: +1 514-843-6411
------------------------------
Business Category: Modern European
Business Name: Bouillon Bilk
No of Reviews: 503
Rating: 4.5
Business Address: 1595 Boulevard Saint-Laurent
Business Contact: +1 514-845-1595
------------------------------
Business Category: Speakeasi

Put your parsed results into a DataFrame

In [20]:
restaurants = []

In [21]:
if bike_stations_df is not None:
    for index, row in bike_stations_df.iterrows():
        results = query_yelp_for_places(row['Latitude'], row['Longitude'], radius = 1000, limit = 50)
        
        for each in results:
            if each['categories']:
                Category = each['categories'][0]['title']
            if each['name']:
                Name = each['name']
            if each['review_count']:
                Count_of_Reviews = each['review_count']
            if each['rating']:
                Rating = each['rating']
            if each['location']['display_address']:
                Address = each['location']['display_address'][0]
            if each['display_phone']:
                Contact = each['display_phone']
                
            restaurants.append({'Station Name': row['Station Name'], 'Station Latitude': row['Latitude'], 'Station Longitude': row['Longitude'], 'Business Category': Category, 'Business Name': Name, 'Rating': Rating, 'Number of Reviews': Count_of_Reviews, 'Address': Address, 'Contact': Contact})

In [22]:
yelp_business_df = pd.DataFrame(restaurants)

In [23]:
yelp_business_df

,Station Name,Station Latitude,Station Longitude,Business Category,Business Name,Rating,Number of Reviews,Address,Contact
0,Gare d'autocars de Montréal (Berri / Ontario),45.516926,-73.564257,Brasseries,Le Saint-Bock,4.0,208,1749 Rue Saint-Denis,+1 514-680-8052
1,Gare d'autocars de Montréal (Berri / Ontario),45.516926,-73.564257,Tapas Bars,L'Amère à Boire,4.0,68,2049 Rue Saint-Denis,+1 514-282-7448
2,Gare d'autocars de Montréal (Berri / Ontario),45.516926,-73.564257,Pizza,Pizzeria Dei Compari,4.0,91,1668 Rue Saint-Denis,+1 514-843-6411
3,Gare d'autocars de Montréal (Berri / Ontario),45.516926,-73.564257,Modern European,Bouillon Bilk,4.5,503,1595 Boulevard Saint-Laurent,+1 514-845-1595
4,Gare d'autocars de Montréal (Berri / Ontario),45.516926,-73.564257,Speakeasies,Le 4e Mur,4.5,99,2021 Rue Saint-Denis,+1 438-396-8947
...,...,...,...,...,...,...,...,...,...
3177,Ste-Catherine / Union,45.503738,-73.569485,Brasseries,Henri - Brasserie Francaise,4.0,44,1240 Phillips Square,+1 514-544-3674
3178,Ste-Catherine / Union,45.503738,-73.569485,Diners,Deville Dinerbar,4.0,609,1425 Rue Stanley,+1 514-281-6556
3179,Ste-Catherine / Union,45.503738,-73.569485,Asian Fusion,MajesThé,4.0,166,2077 Boulevard Robert-Bourassa,+1 514-840-5128
3180,Ste-Catherine / Union,45.503738,-73.569485,Delis,Reuben's Deli & Steakhouse,4.0,674,1116 Rue Sainte-Catherine Ouest,+1 514-866-1029


In [24]:
yelp_business_df.to_csv('yelp_business.csv')

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp API provided me with a lot more data than Four Square API. I was only able to extract three data points from Four Square: The business name, category and location. Other data provided does not provide any value to the current exercise. On the other hand, Yelp offers additional information such as the number of reviews, the rating, type of transactions offered (pick up or delivery), the currency accepted and a phone number to contact the business.

Get the top 10 restaurants according to their rating

In [25]:
Top_10_Restaurants = yelp_business_df.sort_values(by='Rating', ascending = False).head(10)
print(Top_10_Restaurants)

                                        Station Name  Station Latitude  \
1060                               Tolhurst / Fleury         45.544079   
2411                           Ontario / Ville-Marie         45.560020   
155                               Chabanel / du Parc         45.538308   
1609                            6e Avenue / Bélanger         45.554583   
1814        Métro Viau ( Pierre-de-Coubertin / Viau)         45.562219   
162   Métro Henri-Bourassa (Henri-Bourassa / Millen)         45.556751   
1361                                 Duke / Nazareth         45.495722   
2362                                 Hamel / Sauriol         45.561570   
177   Métro Henri-Bourassa (Henri-Bourassa / Millen)         45.556751   
2841                                St-Jacques / Guy         45.491367   

      Station Longitude  Business Category    Business Name  Rating  \
1060         -73.667357  Coffee Roasteries     Café Barista     5.0   
2411         -73.534935             French 